In [1]:
from collections import defaultdict
from os.path import join as pjoin
from time import time
from glob import glob
from typing import Mapping, Any, Optional
import re
import numpy as np
import os
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from textworld import EnvInfos
from gg_pretrained import ggModel
import textworld.gym

In [2]:
import csv
import random

In [3]:
def get_action_phrase_map():
    nutral_command_list = ["wait", "look", "examine", "go north", "go west", "go east", "go south"]
    
    action_to_phrase_map = dict()
    phrase_to_action_map = dict()
    rand_phrase_no = random.randint(0,2)
    with open('State-Action_Phrase_Responses_Superhero.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        line_count = 0
        for row in csv_reader:
            phrase_list = []
            action = row['Action']
            if action not in nutral_command_list:
                phrase_list.append(row['Phrase 1'])
                phrase_list.append(row['Phrase 2'])
                phrase_list.append(row['Phrase 3'])
                phrase = phrase_list[rand_phrase_no]
                action_to_phrase_map[action] = phrase
                phrase_to_action_map[phrase] = action
    
    return action_to_phrase_map, phrase_to_action_map

In [14]:
def get_action_phrase_map(phrase_mode):
    nutral_command_list = ["wait", "look", "examine", "go north", "go west", "go east", "go south"]
    
    action_to_phrase_map = dict()
    phrase_to_action_map = dict()
    with open('State-Action_Phrase_Responses_Superhero.csv', mode='r') as csv_file:
        csv_reader = csv.DictReader(csv_file)
        for row in csv_reader:
            phrase_list = []
            action = row['Action']
            if action not in nutral_command_list:
                if phrase_mode == 'Phrase 1':
                    phrase = row['Phrase 1']
                elif phrase_mode == 'Phrase 2':
                    phrase = row['Phrase 2']
                elif phrase_mode == 'Phrase 3':
                    phrase = row['Phrase 3']
                elif phrase_mode == 'random':
                    rand_phrase_no = random.randint(0,2)
                    phrase_list.append(row['Phrase 1'])
                    phrase_list.append(row['Phrase 2'])
                    phrase_list.append(row['Phrase 3'])
                    phrase = phrase_list[rand_phrase_no]
                
                action_to_phrase_map[action] = phrase
                phrase_to_action_map[phrase] = action
    
    return action_to_phrase_map, phrase_to_action_map


In [ ]:
for phrase_mode in phrase_modes:
    action_to_phrase_pair_map, phrase_to_action_pair_map = get_action_phrase_map()
    agent = NeuralAgent()
    gg_model = ggModel()
    step_size = 750

    print(" =====  Training  ===================================================== ")
    agent.train()  # Tell the agent it should update its parameters.
    start_time = time()
    print(os.path.realpath(game_path))
    play(agent, gg_model, game_path, max_step=step_size, nb_episodes=2500, verbose=False)
    print("Trained in {:.2f} secs".format(time() - start_time))

    print(' =====  Test  ========================================================= ')
    agent.test(method='random')
    play(agent, gg_model, game_path, max_step=step_size)  # Medium level game.

    save_path = "./model/levelMedium_v1_random.npy"
    if not os.path.exists(os.path.dirname(save_path)):
        os.mkdir(os.path.dirname(save_path))


In [23]:
x = [a for a in range(50)]

In [24]:
x

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49]

In [19]:
action_to_phrase_pair_map, phrase_to_action_pair_map = get_action_phrase_map("Phrase 3")

In [20]:
action_to_phrase_pair_map

{'beat the informant': 'The person beat the informant',
 'give the informant some money': 'The person gave the money to the informant',
 'ask the informant about bank robbery': 'The person asks about the bank robbery',
 'drop money': 'The person dropped the money they had',
 'take money': 'The person grabbed the money on the table',
 'shoot the robbers': 'The person shot the robbers',
 'allow the robbers to escape': 'The person allowed the robbers to get away',
 'capture the robbers': 'The person captured the robbers with a rope',
 'convince the robbers to surrender': 'The person convinced the robbers to give up',
 'call the police': 'The person called the cops'}

In [6]:
def preprocess_admissible_commands(commands):
    new_commands = []
    for command in commands:
        if command in action_to_phrase_pair_map:
            new_commands.append(action_to_phrase_pair_map[command])
        else:
            new_commands.append(command)
    return new_commands

In [7]:
# PATH = pjoin(os.path.dirname(__file__), 'textworld_data')
device = torch.device("cpu")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
def get_commands_score(commands):
    score = []
    for command in commands:
        if command in commands_gg_score_map:
            score.append(commands_gg_score_map[command])
        else:
            gg_result = gg_model.get_score(command)
            gg_score = gg_result[0][1]
            score.append(gg_score)
            commands_gg_score_map[command] = gg_score
    score = np.array(score)
    score = score.reshape(1,1,-1)
    score_r = torch.from_numpy(score)
    return score_r

In [9]:
class ActorzCritic(nn.Module):

    eps = 0.01

    def __init__(self, input_size, hidden_size):
        super(ActorzCritic, self).__init__()
        torch.manual_seed(42)  # For reproducibility
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.encoder_gru = nn.GRU(hidden_size, hidden_size)
        self.cmd_encoder_gru = nn.GRU(hidden_size, hidden_size)
        self.state_gru = nn.GRU(hidden_size, hidden_size)

        self.linear_1 = nn.Linear(2 * hidden_size, 2 * hidden_size)
        self.critic = nn.Linear(hidden_size, 1)
        self.actor = nn.Linear(hidden_size * 2, 1)

        # Parameters
        self.state_hidden = torch.zeros(1, 1, hidden_size, device=device)
        self.hidden_size = hidden_size

    def forward(self, obs, commands, command_scores, mode, method):
        input_length, batch_size = obs.size(0), obs.size(1)
        nb_cmds = commands.size(1)

        embedded = self.embedding(obs)
        encoder_output, encoder_hidden = self.encoder_gru(embedded)

        state_output, state_hidden = self.state_gru(encoder_hidden, self.state_hidden)
        self.state_hidden = state_hidden
        state_value = self.critic(state_output)

        # Attention network over the commands.
        cmds_embedding = self.embedding.forward(commands)
        _, cmds_encoding_last_states = self.cmd_encoder_gru.forward(cmds_embedding)  # 1*cmds*hidden

        # Same observed state for all commands.
        cmd_selector_input = torch.stack([state_hidden] * nb_cmds, 2)  # 1*batch*cmds*hidden

        # Same command choices for the whole batch.
        cmds_encoding_last_states = torch.stack([cmds_encoding_last_states] * batch_size, 1)  # 1*batch*cmds*hidden

        # Concatenate the observed state and command encodings.
        input_ = torch.cat([cmd_selector_input, cmds_encoding_last_states], dim=-1)

        # One FC layer
        x = F.relu(self.linear_1(input_))

        # Compute state-action value (score) per command.
        action_state = F.relu(self.actor(x)).squeeze(-1)  # 1 x Batch x cmds
        print("action_state: ", action_state)
        action_state = action_state * command_scores
        print("action_state_1: ", action_state)
        # action_state = F.relu(self.actor(input_)).squeeze(-1)  # 1 x Batch x cmds

        probs = F.softmax(action_state, dim=2)  # 1 x Batch x cmds
        print("probs_0: ", probs)
#         probs = probs * command_scores
#         print("probs_1: ", probs)

        if mode == "train":
            action_index = probs[0].multinomial(num_samples=1).unsqueeze(0)  # 1 x batch x indx
        elif mode == "test":
            if method == 'random':
                action_index = probs[0].multinomial(num_samples=1).unsqueeze(0)  # 1 x batch x indx
            elif method == 'arg-max':
                action_index = probs[0].max(1).indices.unsqueeze(-1).unsqueeze(-1)  # 1 x batch x indx
            elif method == 'eps-soft':
                index = probs[0].max(1).indices.unsqueeze(-1).unsqueeze(-1)
                p = np.random.random()
                if p < (1 - self.eps + self.eps / nb_cmds):
                    action_index = index
                else:
                    while True:
                        tp = np.random.choice(probs[0][0].detach().numpy())
                        if (probs[0][0] == tp).nonzero().unsqueeze(-1) != index:
                            action_index = (probs[0][0] == tp).nonzero().unsqueeze(-1)
                            break

        return action_state, action_index, state_value

    def reset_hidden(self, batch_size):
        self.state_hidden = torch.zeros(1, batch_size, self.hidden_size, device=device)

In [10]:

class NeuralAgent:
    """ Simple Neural Agent for playing TextWorld games. """

    MAX_VOCAB_SIZE = 1000
    UPDATE_FREQUENCY = 10
    LOG_FREQUENCY = 1000
    GAMMA = 0.9

    def __init__(self) -> None:
        self.id2word = ["<PAD>", "<UNK>"]
        self.word2id = {w: i for i, w in enumerate(self.id2word)}

        self.model = ActorzCritic(input_size=self.MAX_VOCAB_SIZE, hidden_size=128)
        self.optimizer = optim.Adam(self.model.parameters(), 0.00003)

    def train(self):
        self.mode = "train"
        self.method = "random"
        self.transitions = []
        self.last_score = 0
        self.no_train_step = 0
        self.stats = {"max": defaultdict(list), "mean": defaultdict(list)}
        self.memo = {"max": defaultdict(list), "mean": defaultdict(list), "mem": defaultdict(list)}
        self.model.reset_hidden(1)

    def test(self, method):
        self.mode = "test"
        self.method = method
        self.model.reset_hidden(1)

    @property
    def infos_to_request(self) -> EnvInfos:
        return EnvInfos(description=True, inventory=True, admissible_commands=True, won=True, lost=True)

    def act(self, obs: str, score: int, done: bool, infos: Mapping[str, Any]) -> Optional[str]:
        # Build agent's observation: feedback + look + inventory.
        input_ = "{}\n{}\n{}".format(obs, infos["description"], infos["inventory"])

        # Tokenize and pad the input and the commands to chose from.
        input_tensor = self._process([input_])
        print("admissible_commands: ", infos["admissible_commands"])
        admissible_commands = preprocess_admissible_commands(infos["admissible_commands"])
        print("preprocessed admissible_commands: ", admissible_commands)
        commands_tensor = self._process(admissible_commands)
        command_scores = get_commands_score(admissible_commands)

        # Get our next action and value prediction.
        outputs, indexes, values = self.model(input_tensor, commands_tensor, command_scores, mode=self.mode, method=self.method)
#         action = infos["admissible_commands"][indexes[0]]
        
        action = admissible_commands[indexes[0]]

        if self.mode == "test":
            if done:
                self.model.reset_hidden(1)
            return action

        self.no_train_step += 1

        if self.transitions:
            reward = score - self.last_score  # Reward is the gain/loss in score.
            self.last_score = score
            if infos["won"]:
                reward += 100
            if infos["lost"]:
                reward -= 100

            self.transitions[-1][0] = reward  # Update reward information.

        self.stats["max"]["score"].append(score)
        self.memo["max"]["score"].append(score)

        if self.no_train_step % self.UPDATE_FREQUENCY == 0:
            # Update model
            returns, advantages = self._discount_rewards(values)

            loss = 0
            for transition, ret, advantage in zip(self.transitions, returns, advantages):
                reward, indexes_, outputs_, values_ = transition

                advantage = advantage.detach()  # Block gradients flow here.
                probs = F.softmax(outputs_, dim=2)
                log_probs = torch.log(probs)
                log_action_probs = log_probs.gather(2, indexes_)
                policy_loss = (log_action_probs * advantage).sum()
                value_loss = ((values_ - ret) ** 2.).sum()
                entropy = (-probs * log_probs).sum()
                loss += 0.5 * value_loss - policy_loss - 0.001 * entropy

                self.memo["mem"]["selected_action_index"].append(indexes_.item())
                self.memo["mem"]["state_val_func"].append(values_.item())
                self.memo["mem"]["advantage"].append(advantage.item())
                self.memo["mem"]["return"].append(ret.item())
                self.memo["mean"]["reward"].append(reward)
                self.memo["mean"]["policy_loss"].append(policy_loss.item())
                self.memo["mean"]["value_loss"].append(value_loss.item())

                self.stats["mean"]["reward"].append(reward)
                self.stats["mean"]["policy_loss"].append(policy_loss.item())
                self.stats["mean"]["value_loss"].append(value_loss.item())
                self.stats["mean"]["entropy"].append(entropy.item())
                self.stats["mean"]["confidence"].append(torch.exp(log_action_probs).item())

            if self.no_train_step % self.LOG_FREQUENCY == 0:
                msg = "{}. ".format(self.no_train_step)
                msg += "  ".join("{}: {:.3f}".format(k, np.mean(v)) for k, v in self.stats["mean"].items())
                msg += "  " + "  ".join("{}: {}".format(k, np.max(v)) for k, v in self.stats["max"].items())
                msg += "  vocab: {}".format(len(self.id2word))
                print(msg)
                self.stats = {"max": defaultdict(list), "mean": defaultdict(list)}

            self.optimizer.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm(self.model.parameters(), 40)
            self.optimizer.step()
            self.optimizer.zero_grad()

            self.transitions = []
            self.model.reset_hidden(1)
        else:
            # Keep information about transitions for Truncated Backpropagation Through Time.
            self.transitions.append([None, indexes, outputs, values])  # Reward will be set on the next call

        if done:
            self.last_score = 0  # Will be starting a new episode. Reset the last score.

        return action

    def _process(self, texts):
        texts = list(map(self._tokenize, texts))
        max_len = max(len(l) for l in texts)
        padded = np.ones((len(texts), max_len)) * self.word2id["<PAD>"]

        for i, text in enumerate(texts):
            padded[i, :len(text)] = text

        padded_tensor = torch.from_numpy(padded).type(torch.long).to(device)
        padded_tensor = padded_tensor.permute(1, 0)  # Batch x Seq => Seq x Batch
        return padded_tensor

    def _tokenize(self, text):
        # Simple tokenizer: strip out all non-alphabetic characters.
        text = re.sub("[^a-zA-Z0-9\- ]", " ", text)
        word_ids = list(map(self._get_word_id, text.split()))
        return word_ids

    def _get_word_id(self, word):
        if word not in self.word2id:
            if len(self.word2id) >= self.MAX_VOCAB_SIZE:
                return self.word2id["<UNK>"]

            self.id2word.append(word)
            self.word2id[word] = len(self.word2id)

        return self.word2id[word]

    def _discount_rewards(self, last_values):
        returns, advantages = [], []
        R = last_values.data
        for t in reversed(range(len(self.transitions))):
            rewards, _, _, values = self.transitions[t]
            R = rewards + self.GAMMA * R
            adv = R - values
            returns.append(R)
            advantages.append(adv)

        return returns[::-1], advantages[::-1]

In [ ]:
game_command = command
if game_command in phrase_to_action_pair_map:
    game_command = phrase_to_action_pair_map[game_command]

In [11]:

def play(agent, path, max_step=50, nb_episodes=10, verbose=True):
    """
        This code uses the cooking agent design in the spaceship game.

        :param agent: the obj of NeuralAgent, a sample object for the agent
        :param path: The path to the game (envo model)
    """

    infos_to_request = agent.infos_to_request
    infos_to_request.max_score = True  # Needed to normalize the scores.

    gamefiles = [path]
    if os.path.isdir(path):
        gamefiles = glob(os.path.join(path, "*.ulx"))

    env_id = textworld.gym.register_games(gamefiles,
                                          request_infos=infos_to_request,
                                          max_episode_steps=max_step)
    env = gym.make(env_id)  # Create a Gym environment to play the text game.

    if verbose:
        if os.path.isdir(path):
            print(os.path.dirname(path), end="")
        else:
            print(os.path.basename(path), end="")

    # Collect some statistics: nb_steps, final reward.
    avg_moves, avg_scores, avg_norm_scores, seed_h = [], [], [], 4567
    for no_episode in range(nb_episodes):
        print("episode: ", no_episode)
        obs, infos = env.reset()  # Start new episode.

        env.env.textworld_env._wrapped_env.seed(seed=seed_h)
        seed_h += 1

        score = 0
        done = False
        nb_moves = 0
        while not done:
            command = agent.act(obs, score, done, infos)
            print("command -> ", command)
            obs, score, done, infos = env.step(command)
            nb_moves += 1
        agent.act(obs, score, done, infos)  # Let the agent know the game is done.

        if verbose:
            print(".", end="")
        avg_moves.append(nb_moves)
        avg_scores.append(score)
        avg_norm_scores.append(score / infos["max_score"])
        print("episode_score: ",score)

    env.close()
    msg = "  \tavg. steps: {:5.1f}; avg. score: {:4.1f} / {}."
    if verbose:
        if os.path.isdir(path):
            print(msg.format(np.mean(avg_moves), np.mean(avg_norm_scores), 1))
        else:
            print(avg_scores)
            print(msg.format(np.mean(avg_moves), np.mean(avg_scores), infos["max_score"]))


In [12]:
game_path = "./tw_games/super_hero_10.ulx"
agent = NeuralAgent()
gg_model = ggModel()
commands_gg_score_map = dict()
step_size = 750

print(" =====  Training  ===================================================== ")
agent.train()  # Tell the agent it should update its parameters.
start_time = time()
# print(os.path.realpath("./games/levelMedium_v1.ulx"))
print(os.path.realpath(game_path))
# play(agent, "./games/levelMedium_v1.ulx", max_step=step_size, nb_episodes=2000, verbose=False)
play(agent, game_path, max_step=step_size, nb_episodes=2000, verbose=False)
print("Trained in {:.2f} secs".format(time() - start_time))

print(' =====  Test  ========================================================= ')
agent.test(method='arg-max')
# play(agent, "./games/levelMedium_v1.ulx", max_step=step_size)  # Medium level game.
play(agent, game_path, max_step=step_size)  # Medium level game.

save_path = "./model/levelMedium_v1_random.npy"
if not os.path.exists(os.path.dirname(save_path)):
    os.mkdir(os.path.dirname(save_path))

np.save(save_path, agent)

 =====  Training  ===================================================== 
/media/nahian/Media/CORGI_Projects/Learning_from_story/LNFStories_gitrepo/Learning_from_stories/tw_games/super_hero_10.ulx
episode:  0
admissible_commands:  ['drop money', 'go east', 'look', 'wait']
preprocessed admissible_commands:  ['She dropped some coins', 'go east', 'look', 'wait']


100%|██████████| 1/1 [00:00<00:00, 16.50it/s]


action_state:  tensor([[[0.0000, 0.0089, 0.0258, 0.0033]]], grad_fn=<SqueezeBackward1>)
action_state_1:  tensor([[[0.0000, 0.0317, 0.0928, 0.0117]]], grad_fn=<MulBackward0>)
probs_0:  tensor([[[0.2415, 0.2493, 0.2650, 0.2443]]], grad_fn=<SoftmaxBackward>)
command ->  look
admissible_commands:  ['drop money', 'go east', 'look', 'wait']
preprocessed admissible_commands:  ['She dropped some coins', 'go east', 'look', 'wait']
action_state:  tensor([[[0.0000, 0.0066, 0.0221, 0.0000]]], grad_fn=<SqueezeBackward1>)
action_state_1:  tensor([[[0.0000, 0.0235, 0.0797, 0.0000]]], grad_fn=<MulBackward0>)
probs_0:  tensor([[[0.2435, 0.2493, 0.2637, 0.2435]]], grad_fn=<SoftmaxBackward>)
command ->  wait
admissible_commands:  ['drop money', 'go east', 'look', 'wait']
preprocessed admissible_commands:  ['She dropped some coins', 'go east', 'look', 'wait']
action_state:  tensor([[[0.0000, 0.0058, 0.0208, 0.0000]]], grad_fn=<SqueezeBackward1>)
action_state_1:  tensor([[[0.0000, 0.0208, 0.0750, 0.0000]]]

100%|██████████| 1/1 [00:00<00:00, 20.97it/s]


action_state:  tensor([[[0.0045, 0.0217, 0.0272, 0.0000, 0.0285, 0.0290, 0.0289, 0.0288,
          0.0297, 0.0256]]], grad_fn=<SqueezeBackward1>)
action_state_1:  tensor([[[-0.0174,  0.0843,  0.1014,  0.0000,  0.1022,  0.1035,  0.1000,
           0.1022,  0.1068,  0.0913]]], grad_fn=<MulBackward0>)
probs_0:  tensor([[[0.0909, 0.1006, 0.1023, 0.0925, 0.1024, 0.1025, 0.1022, 0.1024,
          0.1029, 0.1013]]], grad_fn=<SoftmaxBackward>)
command ->  She dropped some coins
admissible_commands:  ['beat the informant', 'drop money', 'examine informant', 'give the informant some money', 'go east', 'go north', 'go south', 'go west', 'look', 'wait']
preprocessed admissible_commands:  ['He beat the informant mercilessly', 'She dropped some coins', 'examine informant', 'He gave some money for the info he wanted', 'go east', 'go north', 'go south', 'go west', 'look', 'wait']
action_state:  tensor([[[0.0045, 0.0219, 0.0271, 0.0000, 0.0285, 0.0289, 0.0289, 0.0288,
          0.0295, 0.0256]]], grad_

...

command ->  He gave some money for the info he wanted
admissible_commands:  ['beat the informant', 'drop money', 'examine informant', 'give the informant some money', 'go east', 'go north', 'go south', 'go west', 'look', 'wait']
preprocessed admissible_commands:  ['He beat the informant mercilessly', 'She dropped some coins', 'examine informant', 'He gave some money for the info he wanted', 'go east', 'go north', 'go south', 'go west', 'look', 'wait']
action_state:  tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       grad_fn=<SqueezeBackward1>)
action_state_1:  tensor([[[-0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], grad_fn=<MulBackward0>)
probs_0:  tensor([[[0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000]]], grad_fn=<SoftmaxBackward>)
command ->  wait
admissible_commands:  ['beat the informant', 'drop money', 'examine informant', 'give the informant some money', 'go east', 'go north', 'go south', 'go west', 'look', 'wait']
preprocessed a

command ->  go south
admissible_commands:  ['drop money', 'go north', 'look', 'wait']
preprocessed admissible_commands:  ['She dropped some coins', 'go north', 'look', 'wait']
action_state:  tensor([[[0., 0., 0., 0.]]], grad_fn=<SqueezeBackward1>)
action_state_1:  tensor([[[0., 0., 0., 0.]]], grad_fn=<MulBackward0>)
probs_0:  tensor([[[0.2500, 0.2500, 0.2500, 0.2500]]], grad_fn=<SoftmaxBackward>)
episode_score:  0
episode:  36
admissible_commands:  ['drop money', 'go east', 'look', 'wait']
preprocessed admissible_commands:  ['She dropped some coins', 'go east', 'look', 'wait']
action_state:  tensor([[[0., 0., 0., 0.]]], grad_fn=<SqueezeBackward1>)
action_state_1:  tensor([[[0., 0., 0., 0.]]], grad_fn=<MulBackward0>)
probs_0:  tensor([[[0.2500, 0.2500, 0.2500, 0.2500]]], grad_fn=<SoftmaxBackward>)
command ->  go east
admissible_commands:  ['beat the informant', 'drop money', 'examine informant', 'give the informant some money', 'go east', 'go north', 'go south', 'go west', 'look', 'wait

admissible_commands:  ['beat the informant', 'drop money', 'examine informant', 'give the informant some money', 'go east', 'go north', 'go south', 'go west', 'look', 'wait']
preprocessed admissible_commands:  ['He beat the informant mercilessly', 'She dropped some coins', 'examine informant', 'He gave some money for the info he wanted', 'go east', 'go north', 'go south', 'go west', 'look', 'wait']
action_state:  tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       grad_fn=<SqueezeBackward1>)
action_state_1:  tensor([[[-0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]], grad_fn=<MulBackward0>)
probs_0:  tensor([[[0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
          0.1000, 0.1000]]], grad_fn=<SoftmaxBackward>)
command ->  He gave some money for the info he wanted
admissible_commands:  ['beat the informant', 'drop money', 'examine informant', 'give the informant some money', 'go east', 'go north', 'go south', 'go west', 'look', 'wait']
preprocessed admissible_command

episode_score:  0
episode:  39
admissible_commands:  ['drop money', 'go east', 'look', 'wait']
preprocessed admissible_commands:  ['She dropped some coins', 'go east', 'look', 'wait']
action_state:  tensor([[[0., 0., 0., 0.]]], grad_fn=<SqueezeBackward1>)
action_state_1:  tensor([[[0., 0., 0., 0.]]], grad_fn=<MulBackward0>)
probs_0:  tensor([[[0.2500, 0.2500, 0.2500, 0.2500]]], grad_fn=<SoftmaxBackward>)
command ->  go east
admissible_commands:  ['beat the informant', 'drop money', 'examine informant', 'give the informant some money', 'go east', 'go north', 'go south', 'go west', 'look', 'wait']
preprocessed admissible_commands:  ['He beat the informant mercilessly', 'She dropped some coins', 'examine informant', 'He gave some money for the info he wanted', 'go east', 'go north', 'go south', 'go west', 'look', 'wait']
action_state:  tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]],
       grad_fn=<SqueezeBackward1>)
action_state_1:  tensor([[[-0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]

KeyboardInterrupt: 

In [21]:
x = torch.from_numpy(np.array([[[1, 2, 3]]]))

In [22]:
y = torch.from_numpy(np.array([[[2,3,4]]]))

In [23]:
w = x*y

In [25]:
w.shape

torch.Size([1, 1, 3])

In [26]:
z = np.array([1,2,3])

In [27]:
z = z.reshape(1,1,-1)

In [28]:
x = torch.from_numpy(z)

In [29]:
x

tensor([[[1, 2, 3]]])

In [1]:
from gg_pretrained import ggModel

In [2]:
gg_model = ggModel()

In [49]:
desc_1 = "You are in a road. Some mobs are planning to rob a bank. You need to stop them. Go east to the alley. You can find a person in the alley who has information about the roberry. Collect information from him and prevent the robbery."
desc_2 = "This is an alley. There is a person beside the table. He knows about the bank robbery. Do a favor or beat him."
desc_3 = "This is the north bank. Some robbers are going to rob the bank. You can call the police and try to capture them or convince them to surrender. Or you can also shoot them to stop the robbery."

In [32]:
gg_model.get_score(desc_2 + " "+ "The person beat the informant.")

100%|██████████| 1/1 [00:00<00:00,  6.54it/s]


array([[ 0.513862 , -0.9546393]], dtype=float32)

In [31]:
gg_model.get_score(desc_2 + " "+ "The person gave the money to the informant.")

100%|██████████| 1/1 [00:00<00:00,  5.68it/s]


array([[-3.3349154,  3.507331 ]], dtype=float32)

In [30]:
gg_model.get_score(desc_2 + " "+ "The person asks about the bank robbery.")

100%|██████████| 1/1 [00:00<00:00,  6.99it/s]


array([[-3.304716 ,  3.6697328]], dtype=float32)

In [29]:
gg_model.get_score("The person shot the robbers.")

100%|██████████| 1/1 [00:00<00:00,  7.78it/s]


array([[ 3.4661903, -3.7697582]], dtype=float32)

In [25]:
gg_model.get_score(desc_3 + " "+ "The person shot the robbers.")

100%|██████████| 1/1 [00:00<00:00,  5.76it/s]


array([[ 2.8910067, -3.5295725]], dtype=float32)

In [28]:
gg_model.get_score(desc_3 + " "+ "The person convinced the robbers to give up.")

100%|██████████| 1/1 [00:00<00:00, 15.90it/s]


array([[-0.3063048,  0.8177075]], dtype=float32)

In [ ]:
gg_model.get_score(desc_3 + " "+ "The person called the cops.")

In [7]:
gg_model.get_score("This is the north bank. Some robbers are going to rob the bank. You can call the police and try to capture them or convince them to surrender. Or you can also shoot them to stop the robbery. There is a robber on the floor. call the police")

100%|██████████| 1/1 [00:00<00:00, 10.90it/s]


array([[ 1.10023   , -0.91799563]], dtype=float32)

In [8]:
gg_model.get_score("This is the north bank. Some robbers are going to rob the bank. You can call the police and try to capture them or convince them to surrender. Or you can also shoot them to stop the robbery. There is a robber on the floor. shoot the robber")

100%|██████████| 1/1 [00:00<00:00, 11.10it/s]


array([[ 1.7523423, -1.5920073]], dtype=float32)

In [38]:
gg_model.get_score(desc_3 + " "+ "Negotiating was something Bill wanted to try.")

100%|██████████| 1/1 [00:00<00:00,  6.62it/s]


array([[-1.5649904,  2.2837117]], dtype=float32)

In [43]:
gg_model.get_score(desc_2 + " "+ "The man asked for some details about the impending job.")

100%|██████████| 1/1 [00:00<00:00, 11.28it/s]


array([[-3.1111462,  3.4574401]], dtype=float32)

In [44]:
gg_model.get_score(desc_2 + " "+ "The informant was beaten by the man.")

100%|██████████| 1/1 [00:00<00:00,  6.56it/s]


array([[-2.175722 ,  2.3467505]], dtype=float32)

In [53]:
gg_model.get_score(desc_2 + " "+ "The informant was beaten by the man.")

100%|██████████| 1/1 [00:00<00:00,  6.35it/s]


array([[-0.7177407 ,  0.11902849]], dtype=float32)

In [45]:
gg_model.get_score(desc_3 + " "+ "He opened fire on the robbers.")

100%|██████████| 1/1 [00:00<00:00,  7.23it/s]


array([[ 3.0255797, -3.6448135]], dtype=float32)

In [54]:
gg_model.get_score(desc_3 + " "+ "He took out his phone and dialed 911.")

100%|██████████| 1/1 [00:00<00:00,  9.39it/s]


array([[-0.16965811,  0.6919911 ]], dtype=float32)

In [47]:
gg_model.get_score(desc_3 + " "+ "He did his best to negotiate with the men.")

100%|██████████| 1/1 [00:00<00:00, 11.87it/s]


array([[-2.0801744,  2.5675168]], dtype=float32)

In [ ]:
This is the north bank. Some robbers are going to rob the bank. You can call the police and try to capture them or convince them to surrender. Or you can also shoot them to stop the robbery.

In [102]:
play_desc1 = "You are in the Locker room with your team mates. Collect your game gears and wear them before the meeting. Be helpful and dont disturb others."
play_desc2 = "You are in the meeting room. Don't miss the meeting. Join the meeting and try to share your thought. After the meeting go to east to the field."

In [99]:
gg_model.get_score("James interrupted them before they left." + " "+play_desc1)

100%|██████████| 1/1 [00:00<00:00, 13.95it/s]


array([[-2.7654066,  3.3455422]], dtype=float32)

In [94]:
gg_model.get_score("James interrupted them before they left.")

100%|██████████| 1/1 [00:00<00:00, 13.41it/s]


array([[ 1.1737088, -2.0962436]], dtype=float32)

In [95]:
gg_model.get_score("Jude just couldnt stand meetings so he didnt attend.")

100%|██████████| 1/1 [00:00<00:00, 13.27it/s]


array([[ 1.937963 , -1.8380834]], dtype=float32)

In [97]:
gg_model.get_score(play_desc2 + "\n " +"Jude just couldnt stand meetings so he didnt attend.")

100%|██████████| 1/1 [00:00<00:00, 12.08it/s]


array([[-3.2380831,  3.900918 ]], dtype=float32)

In [103]:
play_desc_3 = "You are in the Locker room with your team mates. You will find your game gears here. Collect the game gears and wear them before the meeting. Be helpful and dont disturb others. After you are done here, go to the meeting room."
play_desc_4 = "You are in the meeting room. A meeting is going to start to finalize the game plan. Join the meeting and try to share your thought. After the meeting go to east to the field."

In [104]:
gg_model.get_score(play_desc_3 + " "+ "James interrupted them before they left.")

100%|██████████| 1/1 [00:00<00:00, 12.10it/s]


array([[-2.9943168,  3.5765867]], dtype=float32)

In [13]:
import json

In [15]:
json.load(open('image_data.json'))

[{'width': 800,
  'url': 'https://cs.stanford.edu/people/rak248/VG_100K_2/1.jpg',
  'height': 600,
  'image_id': 1,
  'coco_id': None,
  'flickr_id': None},
 {'width': 800,
  'url': 'https://cs.stanford.edu/people/rak248/VG_100K/2.jpg',
  'height': 600,
  'image_id': 2,
  'coco_id': None,
  'flickr_id': None},
 {'width': 640,
  'url': 'https://cs.stanford.edu/people/rak248/VG_100K/3.jpg',
  'height': 480,
  'image_id': 3,
  'coco_id': None,
  'flickr_id': None},
 {'width': 640,
  'url': 'https://cs.stanford.edu/people/rak248/VG_100K/4.jpg',
  'height': 480,
  'image_id': 4,
  'coco_id': None,
  'flickr_id': None},
 {'width': 800,
  'url': 'https://cs.stanford.edu/people/rak248/VG_100K/5.jpg',
  'height': 600,
  'image_id': 5,
  'coco_id': None,
  'flickr_id': None},
 {'width': 800,
  'url': 'https://cs.stanford.edu/people/rak248/VG_100K/6.jpg',
  'height': 600,
  'image_id': 6,
  'coco_id': None,
  'flickr_id': None},
 {'width': 800,
  'url': 'https://cs.stanford.edu/people/rak248/VG_1

In [ ]:
with open('/data/genome/image_data.json') as f:
    for item in json.load(f):
        image_id = int(item['image_id'])
        filepath = os.path.join('/data/genome/', item['url'].split('rak248/')[-1])
        split.append((filepath,image_id))    

In [2]:
action_cycle_map = dict()
action_cycle_map["go north"] = "go east"

In [3]:
def has_cycle(prev_cmd, cur_cmd):
    cycle = False
    if prev_cmd in action_cycle_map:
        if action_cycle_map[prev_cmd] == cur_cmd:
            cycle = True
            
    return cycle
        
        

In [6]:
has_cycle("go north 1", "go east")

False

In [37]:
file1 = open('myfile.txt', 'w') 

In [38]:
s = "abcd"

In [39]:
file1.write("test: {}".format(s))
file1.write("test: {}\n".format(s))

11

In [40]:
file1.close()

In [55]:
a = "asdfsafd "

In [57]:
b = a.strip()

In [58]:
b

'asdfsafd'

In [ ]:
def get_description(description_str):
    desc_1 = "You are in a road. Some mobs are planning to rob a bank. You need to stop them. Go east to the alley. You can find a person in the alley who has information about the roberry. Collect information from him and prevent the robbery."
    desc_2 = "This is an alley. There is a person beside the table. He knows about the bank robbery. Do a favor or beat him"
    desc_3 = "This is the north bank. Some robbers are going to rob the bank. You can call the police and try to capture them or convince them to surrender. Or you can also shoot them to stop the robbery."
    desc = description_str
    if "You are in a road." in description_str:
        desc = desc_1
    elif "This is an alley" in description_str:
        desc = desc_2
    elif "This is the north bank" in description_str:
        desc = desc_3
    return desc